In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import re
import datetime as dt

from pybaseball import *

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

import psycopg2
import pandas.io.sql as psql
from sqlalchemy import create_engine

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df_18 = pd.read_pickle('../pickles/2018_games_labeled_1.pkl')
df_17 = pd.read_pickle('../pickles/2017_games_labeled_1.pkl')
df_16 = pd.read_pickle('../pickles/2016_games_labeled_1.pkl')
df_15 = pd.read_pickle('../pickles/2015_games_labeled_1.pkl')
merge = pd.read_pickle('../pickles/labeled_games_agg.pkl')

In [9]:
sc_18 = pd.read_pickle('../pickles/sc_2018_unclean.pkl')
sc_17 = pd.read_pickle('../pickles/sc_2017_unclean.pkl')
sc_16 = pd.read_pickle('../pickles/sc_2016_unclean.pkl')
sc_15 = pd.read_pickle('../pickles/sc_2015_unclean.pkl')
total = pd.read_pickle('../pickles/labeled_games_agg_sc.pkl')

In [5]:
sc_18

,#days,2b,3b,ab,age,appearance_yr,babip,bb,bf,cs,date,days,days_preceeding,days_rest,enddate,er,era,first_x,first_y,g,gb/fb,gdp,gs,h,hbp,hr,ibb,injured?,injury,ip,key_bbref_x,key_bbref_y,key_fangraphs_x,key_fangraphs_y,key_mlbam,key_retro_x,key_retro_y,l,last_x,last_y,ld,lev,list,mlb_played_first_x,mlb_played_first_y,mlb_played_last_x,mlb_played_last_y,moneyearned,name_first_x,name_first_y,name_last_x,name_last_y,name_x,name_y,pit,po,posititon,pu,r,sb,sf,so,so/w,so9,startdate,stl,str,sts,sv,team,tm,w,whip,year,fb_count,p_throws_x,fb_release_speed,fb_percieved_speed,fb_spin_rate,fb_avg_x,fb_avg_z,fb_avg_extension,fb_x_movement,fb_z_movement,os_count,p_throws_y,os_release_speed,os_percieved_speed,os_spin_rate,os_avg_x,os_avg_z,os_avg_extension,os_x_movement,os_z_movement
0,368.0,0.0,0.0,23.0,45.0,1,0.333,1.0,24.0,0.0,2018-04-02,9.0,144,182.0,9/1,1.0,1.500000,bartolocolon,bartolo,1.0,0.55,1.0,1.0,7.0,0.0,1.0,0.0,0.0,Back,6.0,colonba01,colonba01,375.0,375.0,112526.0,colob001,colob001,NaN,bartolocolon,colon,0.10,MLB-AL,10,1997.0,1997.0,2018.0,2018.0,84681,bartolo,bartolo,colon,colon,bartolocolon,bartolocolon,89.0,0.0,SP,0.05,1.0,0.0,0.0,4.0,4.00,6.0,2018-08-24,0.21,0.70,0.07,NaN,TEX,Texas,NaN,1.333,2018.0,74,R,87.3514,85.375,2160.84,-1.39738,5.77241,5.16942,-1.38904,0.748401,15,R,80.9267,79.1891,1917.8,-1.44304,5.75545,5.10453,-0.63362,0.509173
1,362.0,0.0,0.0,7.0,45.0,2,0.333,0.0,7.0,0.0,2018-04-08,9.0,138,6.0,9/1,0.0,0.000000,bartolocolon,bartolo,1.0,1.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Back,2.0,colonba01,colonba01,375.0,375.0,112526.0,colob001,colob001,NaN,bartolocolon,colon,0.00,MLB-AL,10,1997.0,1997.0,2018.0,2018.0,84681,bartolo,bartolo,colon,colon,bartolocolon,bartolocolon,29.0,0.0,SP,0.00,0.0,0.0,0.0,4.0,NaN,18.0,2018-08-24,0.24,0.72,0.14,NaN,TEX,Texas,NaN,0.500,2018.0,27,R,87.6704,85.1251,2142.7,-1.27045,5.55289,5.13315,-1.43878,0.898693,2,R,81.7,79.5365,1974,-1.40445,5.46085,4.954,-0.39145,0.36785
2,360.0,0.0,0.0,9.0,45.0,3,0.250,0.0,10.0,0.0,2018-04-10,9.0,136,2.0,9/1,1.0,3.000000,bartolocolon,bartolo,1.0,0.38,0.0,0.0,2.0,0.0,0.0,0.0,0.0,Back,3.0,colonba01,colonba01,375.0,375.0,112526.0,colob001,colob001,NaN,bartolocolon,colon,0.25,MLB-AL,10,1997.0,1997.0,2018.0,2018.0,84681,bartolo,bartolo,colon,colon,bartolocolon,bartolocolon,29.0,0.0,SP,0.00,1.0,0.0,1.0,2.0,NaN,6.0,2018-08-24,0.14,0.66,0.10,NaN,TEX,Texas,NaN,0.667,2018.0,25,R,87.716,85.5084,2199.88,-1.42624,5.51528,5.11032,-1.28661,0.810228,4,R,81.975,79.9275,2093,-1.53535,5.51748,5.053,-0.382,0.84215
3,355.0,1.0,0.0,23.0,45.0,4,0.059,1.0,25.0,0.0,2018-04-15,9.0,131,5.0,9/1,1.0,1.170000,bartolocolon,bartolo,1.0,0.59,0.0,1.0,1.0,0.0,0.0,0.0,0.0,Back,7.2,colonba01,colonba01,375.0,375.0,112526.0,colob001,colob001,NaN,bartolocolon,colon,0.18,MLB-AL,10,1997.0,1997.0,2018.0,2018.0,84681,bartolo,bartolo,colon,colon,bartolocolon,bartolocolon,96.0,0.0,SP,0.00,1.0,0.0,1.0,7.0,7.00,8.2,2018-08-24,0.24,0.69,0.08,NaN,TEX,Texas,NaN,0.261,2018.0,83,R,88.9675,86.7409,2218.12,-1.29161,5.44329,5.23055,-1.42098,0.868841,13,R,82.7538,81.0304,2134.23,-1.34288,5.48426,5.19177,-0.341485,0.514208
4,349.0,0.0,0.0,22.0,45.0,5,0.300,1.0,25.0,1.0,2018-04-21,9.0,125,6.0,9/1,4.0,6.350000,bartolocolon,bartolo,1.0,0.52,0.0,1.0,7.0,1.0,1.0,0.0,0.0,Back,5.2,colonba01,colonba01,375.0,375.0,112526.0,colob001,colob001,NaN,bartolocolon,colon,0.24,MLB-AL,10,1997.0,1997.0,2018.0,2018.0,84681,bartolo,bartolo,colon,colon,bartolocolon,bartolocolon,85.0,0.0,SP,0.00,4.0,0.0,1.0,2.0,2.00,3.2,2018-08-24,0.21,0.69,0.08,NaN,TEX,Texas,NaN,1.412,2018.0,66,R,86.897,84.7244,2113.5,-1.31265,5.50848,5.11964,-1.4087,0.86107,12,R,80.4833,78.5709,1910.5,-1.32747,5.57015,5.004,-0.610125,0.603633
5,342.0,1.0,1.0,27.0,45.0,6,0.174,0.0,27.0,0.0,2018-04-28,9.0,118,7.0,9/1,3.0,3.860000,bartolocolon,bartolo,1.0,0.44,0.0,1.0,6.0,0.0,2.0,0.0,0.0,Back,7.0,colonba01,colonba01,375.0,375.0,112526.0,colob001,colob001,NaN,bartolocolon,colon,0.16,MLB-AL,10,1997.0,1997.0,2018.0,2018.0,84681,bartolo,bartolo,colon,colon,bartolocolon,bartolocolon,98

In [ ]:
conn = psycopg2.connect(dbname='statcast', user='taylorphillips', host='localhost')
cur = conn.cursor()

fb_frame = pd.DataFrame()
os_frame = pd.DataFrame()

id_list = total['key_mlbam'].unique().tolist()

for id in id_list:
    fastballs = psql.read_sql('''
    SELECT pitcher as key_mlbam, COUNT(pitch_type) as {0}_count, AVG(release_speed) as {0}_release_speed,
    AVG(effective_speed) as {0}_percieved_speed, AVG(release_spin_rate) as {0}_spin_rate, 
    AVG(release_pos_x) as {0}_avg_x, AVG(release_pos_z) as {0}_avg_z, AVG(release_extension) as {0}_avg_extension,
    AVG(pfx_x) as {0}_x_movement, AVG(pfx_z) as {0}_z_movement 
    FROM pitches 
    WHERE pitcher = {1} AND pitch_type IN ('FF','FT', 'SI')
    GROUP BY pitcher
    ORDER BY pitcher
    '''.format('hist_fb', id), conn)

    offspeed = psql.read_sql('''
    SELECT pitcher as key_mlbam, COUNT(pitch_type) as {0}_count, AVG(release_speed) as {0}_release_speed,
    AVG(effective_speed) as {0}_percieved_speed, AVG(release_spin_rate) as {0}_spin_rate, 
    AVG(release_pos_x) as {0}_avg_x, AVG(release_pos_z) as {0}_avg_z, AVG(release_extension) as {0}_avg_extension,
    AVG(pfx_x) as {0}_x_movement, AVG(pfx_z) as {0}_z_movement 
    FROM pitches 
    WHERE pitcher = {1} AND pitch_type IN ('FC', 'SL', 'FS', 'CH', 'KC', 'CU', 'FO', 'KN', 'SC', 'EP')
    GROUP BY pitcher
    ORDER BY pitcher
    '''.format('hist_os', id), conn)
    

    fb_frame = fb_frame.append(fastballs)
    os_frame = os_frame.append(offspeed)
#     merge_frame = fastballs.merge(offspeed, on='key_mlbam')
#     return_frame = return_frame.append(merge_frame)

In [43]:
conn = psycopg2.connect(dbname='statcast', user='taylorphillips', host='localhost')
cur = conn.cursor()

return_frame1 = pd.DataFrame()
fb_frame = pd.DataFrame()
os_frame = pd.DataFrame()

id_list = total['key_mlbam'].unique().tolist()

for id in id_list:
    fastballs = psql.read_sql('''
    SELECT pitcher as key_mlbam, COUNT(pitch_type) as {0}_count, AVG(release_speed) as {0}_release_speed,
    AVG(effective_speed) as {0}_percieved_speed, AVG(release_spin_rate) as {0}_spin_rate, 
    AVG(release_pos_x) as {0}_avg_x, AVG(release_pos_z) as {0}_avg_z, AVG(release_extension) as {0}_avg_extension,
    AVG(pfx_x) as {0}_x_movement, AVG(pfx_z) as {0}_z_movement 
    FROM pitches 
    WHERE pitcher = {1} AND pitch_type IN ('FF','FT', 'SI')
    GROUP BY pitcher
    ORDER BY pitcher
    '''.format('hist_fb', id), conn)

#     fastballs = psql.read_sql('''
#     SELECT COUNT(pitch_type) as {0}_count, game_date as date, p_throws
#     FROM pitches 
#     WHERE pitcher = {1} AND pitch_type IN ('FF','FT', 'SI')
#     GROUP BY date, p_throws
#     ORDER BY date
#     '''.format('fb', id), conn)
    
    offspeed = psql.read_sql('''
    SELECT pitcher as key_mlbam, COUNT(pitch_type) as {0}_count, AVG(release_speed) as {0}_release_speed,
    AVG(effective_speed) as {0}_percieved_speed, AVG(release_spin_rate) as {0}_spin_rate, 
    AVG(release_pos_x) as {0}_avg_x, AVG(release_pos_z) as {0}_avg_z, AVG(release_extension) as {0}_avg_extension,
    AVG(pfx_x) as {0}_x_movement, AVG(pfx_z) as {0}_z_movement 
    FROM pitches 
    WHERE pitcher = {1} AND pitch_type IN ('FC', 'SL', 'FS', 'CH', 'KC', 'CU', 'FO', 'KN', 'SC', 'EP')
    GROUP BY pitcher
    ORDER BY pitcher
    '''.format('hist_os', id), conn)
    
#     offspeed = psql.read_sql('''
#     SELECT COUNT(pitch_type) as {0}_count, game_date as date, p_throws
#     FROM pitches 
#     WHERE pitcher = {1} AND pitch_type IN ('FF','FT', 'SI')
#     GROUP BY date, p_throws
#     ORDER BY date
#     '''.format('fb', id), conn)
    
#     frame = sc_18[sc_18['key_mlbam'] == id]
    
#     fb_frame = frame.merge(fastballs, on='key_mlbam', how='left')
#     merge_frame = fb_frame.merge(offspeed, on='key_mlbam', how='left')
#     return_frame = return_frame.append(merge_frame)
    fb_frame = fb_frame.append(fastballs)
    os_frame = os_frame.append(offspeed)
#     merge_frame = fastballs.merge(offspeed, on='key_mlbam')
#     return_frame = return_frame.append(merge_frame)

In [53]:
merge1.to_pickle('../pickles/sc_hist.pkl')
return_frame

,hist_fb_avg_extension,hist_fb_avg_x,hist_fb_avg_z,hist_fb_count,hist_fb_percieved_speed,hist_fb_release_speed,hist_fb_spin_rate,hist_fb_x_movement,hist_fb_z_movement,hist_os_avg_extension,hist_os_avg_x,hist_os_avg_z,hist_os_count,hist_os_percieved_speed,hist_os_release_speed,hist_os_spin_rate,hist_os_x_movement,hist_os_z_movement,key_mlbam
0,5.262154,-1.331322,5.707565,8551,86.265537,88.212583,2148.405050,-1.203380,0.998063,5.126860,-1.467546,5.665911,1624,79.950319,81.785151,1980.118786,-0.496410,0.636257,112526.0
0,6.062608,-1.569735,6.420903,444,93.090395,93.770495,2052.904878,-1.168838,1.292336,5.909676,-1.661313,6.533256,154,85.632754,86.373377,1972.307143,-0.315562,0.788430,115629.0
0,5.447056,2.621224,5.618119,54,83.061519,84.277778,2198.074074,1.356790,1.302897,4.913210,2.434560,6.014652,65,77.108500,78.690769,2206.180328,0.060359,0.575048,136600.0
0,5.968527,2.314636,5.959853,301,88.195092,88.781063,2032.142857,0.846146,1.384395,5.491444,2.328470,6.063357,318,77.195853,78.305346,2057.060498,-0.174267,0.237148,150116.0
0,5.841414,-2.136665,6.520854,74,91.106057,91.771622,2241.285714,-0.779758,1.457634,5.463596,-2.283923,6.400510,55,83.670885,84.640000,2219.897959,0.049348,0.245267,150274.0
0,6.111399,-2.504835,5.835588,459,87.671034,88.302179,1782.709677,-1.525396,0.713049,6.021669,-2.553236,5.794250,314,80.616823,80.596178,1587.302817,-0.593044,0.556129,150302.0
0,6.588302,-1.823464,5.918232,1540,91.805808,91.553182,2009.191845,-1.106436,0.910834,6.085209,-1.987720,5.849725,950,82.761909,83.247158,1941.998918,0.114306,-0.262695,150359.0
0,6.522598,0.964147,6.333970,87,84.401839,84.252874,2068.931034,0.462450,1.740010,6.448524,1.105922,6.404273,63,72.002508,72.257143,1971.523810,0.234440,-0.258962,217096.0
0,5.570074,-1.545845,5.277807,1004,87.626304,88.700000,2039.038126,-1.164730,0.601575,5.338461,-1.540103,5.242420,844,79.395071,80.620972,1956.998534,0.080744,0.032925,218596.0
0,6.013837,-1.285595,5.824657,98,89.538929,90.573469,2256.122449,-0.956494,1.323192,5.744500,-1.352730,5.877660,20,84.821250,86.055000,1900.684211,-0.300340,0.758928,234194.0


In [44]:
fb_frame

,key_mlbam,hist_fb_count,hist_fb_release_speed,hist_fb_percieved_speed,hist_fb_spin_rate,hist_fb_avg_x,hist_fb_avg_z,hist_fb_avg_extension,hist_fb_x_movement,hist_fb_z_movement
0,112526.0,8551,88.212583,86.265537,2148.405050,-1.331322,5.707565,5.262154,-1.203380,0.998063
0,115629.0,444,93.770495,93.090395,2052.904878,-1.569735,6.420903,6.062608,-1.168838,1.292336
0,136600.0,54,84.277778,83.061519,2198.074074,2.621224,5.618119,5.447056,1.356790,1.302897
0,150116.0,301,88.781063,88.195092,2032.142857,2.314636,5.959853,5.968527,0.846146,1.384395
0,150274.0,74,91.771622,91.106057,2241.285714,-2.136665,6.520854,5.841414,-0.779758,1.457634
0,150302.0,459,88.302179,87.671034,1782.709677,-2.504835,5.835588,6.111399,-1.525396,0.713049
0,150359.0,1540,91.553182,91.805808,2009.191845,-1.823464,5.918232,6.588302,-1.106436,0.910834
0,217096.0,87,84.252874,84.401839,2068.931034,0.964147,6.333970,6.522598,0.462450,1.740010
0,218596.0,1004,88.700000,87.626304,2039.038126,-1.545845,5.277807,5.570074,-1.164730,0.601575
0,234194.0,98,90.573469,89.538929,2256.122449,-1.285595,5.824657,6.013837,-0.956494,1.323192


In [45]:
os_frame

,key_mlbam,hist_os_count,hist_os_release_speed,hist_os_percieved_speed,hist_os_spin_rate,hist_os_avg_x,hist_os_avg_z,hist_os_avg_extension,hist_os_x_movement,hist_os_z_movement
0,112526.0,1624,81.785151,79.950319,1980.118786,-1.467546,5.665911,5.126860,-0.496410,0.636257
0,115629.0,154,86.373377,85.632754,1972.307143,-1.661313,6.533256,5.909676,-0.315562,0.788430
0,136600.0,65,78.690769,77.108500,2206.180328,2.434560,6.014652,4.913210,0.060359,0.575048
0,150116.0,318,78.305346,77.195853,2057.060498,2.328470,6.063357,5.491444,-0.174267,0.237148
0,150274.0,55,84.640000,83.670885,2219.897959,-2.283923,6.400510,5.463596,0.049348,0.245267
0,150302.0,314,80.596178,80.616823,1587.302817,-2.553236,5.794250,6.021669,-0.593044,0.556129
0,150359.0,950,83.247158,82.761909,1941.998918,-1.987720,5.849725,6.085209,0.114306,-0.262695
0,217096.0,63,72.257143,72.002508,1971.523810,1.105922,6.404273,6.448524,0.234440,-0.258962
0,218596.0,844,80.620972,79.395071,1956.998534,-1.540103,5.242420,5.338461,0.080744,0.032925
0,234194.0,20,86.055000,84.821250,1900.684211,-1.352730,5.877660,5.744500,-0.300340,0.758928


In [50]:
merge1 = fb_frame.merge(os_frame, on='key_mlbam', how='outer')
merge1

,key_mlbam,hist_fb_count,hist_fb_release_speed,hist_fb_percieved_speed,hist_fb_spin_rate,hist_fb_avg_x,hist_fb_avg_z,hist_fb_avg_extension,hist_fb_x_movement,hist_fb_z_movement,hist_os_count,hist_os_release_speed,hist_os_percieved_speed,hist_os_spin_rate,hist_os_avg_x,hist_os_avg_z,hist_os_avg_extension,hist_os_x_movement,hist_os_z_movement
0,112526.0,8551,88.212583,86.265537,2148.405050,-1.331322,5.707565,5.262154,-1.203380,0.998063,1624,81.785151,79.950319,1980.118786,-1.467546,5.665911,5.126860,-0.496410,0.636257
1,115629.0,444,93.770495,93.090395,2052.904878,-1.569735,6.420903,6.062608,-1.168838,1.292336,154,86.373377,85.632754,1972.307143,-1.661313,6.533256,5.909676,-0.315562,0.788430
2,136600.0,54,84.277778,83.061519,2198.074074,2.621224,5.618119,5.447056,1.356790,1.302897,65,78.690769,77.108500,2206.180328,2.434560,6.014652,4.913210,0.060359,0.575048
3,150116.0,301,88.781063,88.195092,2032.142857,2.314636,5.959853,5.968527,0.846146,1.384395,318,78.305346,77.195853,2057.060498,2.328470,6.063357,5.491444,-0.174267,0.237148
4,150274.0,74,91.771622,91.106057,2241.285714,-2.136665,6.520854,5.841414,-0.779758,1.457634,55,84.640000,83.670885,2219.897959,-2.283923,6.400510,5.463596,0.049348,0.245267
5,150302.0,459,88.302179,87.671034,1782.709677,-2.504835,5.835588,6.111399,-1.525396,0.713049,314,80.596178,80.616823,1587.302817,-2.553236,5.794250,6.021669,-0.593044,0.556129
6,150359.0,1540,91.553182,91.805808,2009.191845,-1.823464,5.918232,6.588302,-1.106436,0.910834,950,83.247158,82.761909,1941.998918,-1.987720,5.849725,6.085209,0.114306,-0.262695
7,217096.0,87,84.252874,84.401839,2068.931034,0.964147,6.333970,6.522598,0.462450,1.740010,63,72.257143,72.002508,1971.523810,1.105922,6.404273,6.448524,0.234440,-0.258962
8,218596.0,1004,88.700000,87.626304,2039.038126,-1.545845,5.277807,5.570074,-1.164730,0.601575,844,80.620972,79.395071,1956.998534,-1.540103,5.242420,5.338461,0.080744,0.032925
9,234194.0,98,90.573469,89.538929,2256.122449,-1.285595,5.824657,6.013837,-0.956494,1.323192,20,86.055000,84.821250,1900.684211,-1.352730,5.877660,5.744500,-0.300340,0.758928


In [52]:
# merge1[merge1['hist_fb_count'].isnull()]


,key_mlbam,hist_fb_count,hist_fb_release_speed,hist_fb_percieved_speed,hist_fb_spin_rate,hist_fb_avg_x,hist_fb_avg_z,hist_fb_avg_extension,hist_fb_x_movement,hist_fb_z_movement,hist_os_count,hist_os_release_speed,hist_os_percieved_speed,hist_os_spin_rate,hist_os_avg_x,hist_os_avg_z,hist_os_avg_extension,hist_os_x_movement,hist_os_z_movement
1276,424325.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,71.557895,69.195000,1737.000000,-1.255141,6.677735,4.177875,-0.692526,1.382575
1277,425543.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,84.093750,82.797200,2326.333333,-2.711120,6.200653,5.775933,-0.580552,1.394235
1278,425772.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,80.995000,78.716100,1862.950000,0.147160,6.574780,4.803450,-0.153820,1.659345
1279,430404.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,69.465000,67.201050,1571.277778,-1.862660,6.890830,4.421650,-0.787006,1.317498
1280,430605.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25,82.308000,80.628000,1892.291667,-2.784020,6.000968,4.905958,-0.860514,1.276835
1281,430947.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,69.915385,66.966538,1764.846154,-1.845969,5.358246,4.022231,-0.439969,0.726300
1282,433217.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,71.450000,68.603750,1874.750000,-2.481775,5.834512,4.076125,-1.107613,1.238200
1283,434540.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,73.952941,72.490059,1552.058824,1.716359,7.287653,5.349824,0.619617,1.362433
1284,434563.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,80.420000,78.143133,1818.466667,-1.480580,6.116067,4.894133,-0.696137,1.681870
1285,434778.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,82.912500,82.107500,2288.437500,-2.577994,5.657069,5.625063,-0.139550,0.850625


In [34]:
conn = psycopg2.connect(dbname='statcast', user='taylorphillips', host='localhost')
cur = conn.cursor()

fastballs = psql.read_sql('''
    SELECT pitcher as key_mlbam, COUNT(pitch_type) as {0}_count, AVG(release_speed) as {0}_release_speed,
    AVG(effective_speed) as {0}_percieved_speed, AVG(release_spin_rate) as {0}_spin_rate, 
    AVG(release_pos_x) as {0}_avg_x, AVG(release_pos_z) as {0}_avg_z, AVG(release_extension) as {0}_avg_extension,
    AVG(pfx_x) as {0}_x_movement, AVG(pfx_z) as {0}_z_movement 
    FROM pitches 
    WHERE pitcher = {1} AND pitch_type IN ('FF','FT', 'SI')
    GROUP BY pitcher
    ORDER BY pitcher
    '''.format('hist_fb', 546276.0), conn)

offspeed = psql.read_sql('''
    SELECT pitcher as key_mlbam, COUNT(pitch_type) as {0}_count, AVG(release_speed) as {0}_release_speed,
    AVG(effective_speed) as {0}_percieved_speed, AVG(release_spin_rate) as {0}_spin_rate, 
    AVG(release_pos_x) as {0}_avg_x, AVG(release_pos_z) as {0}_avg_z, AVG(release_extension) as {0}_avg_extension,
    AVG(pfx_x) as {0}_x_movement, AVG(pfx_z) as {0}_z_movement 
    FROM pitches 
    WHERE pitcher = {1} AND pitch_type IN ('FC', 'SL', 'FS', 'CH', 'KC', 'CU', 'FO', 'KN', 'SC', 'EP')
    GROUP BY pitcher
    ORDER BY pitcher
    '''.format('hist_os', 546276.0), conn)


In [37]:
fb_frame.append(fastballs)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,#days,2b,3b,ab,age,appearance_yr,babip,bb,bf,cs,date,days,days_preceeding,days_rest,enddate,er,era,fb_avg_extension,fb_avg_x,fb_avg_z,fb_count,fb_percieved_speed,fb_release_speed,fb_spin_rate,fb_x_movement,fb_z_movement,first_x,first_y,g,gb/fb,gdp,gs,h,hbp,hist_fb_avg_extension,hist_fb_avg_x,hist_fb_avg_z,hist_fb_count,hist_fb_percieved_speed,hist_fb_release_speed,hist_fb_spin_rate,hist_fb_x_movement,hist_fb_z_movement,hr,ibb,injured?,injury,ip,key_bbref_x,key_bbref_y,key_fangraphs_x,key_fangraphs_y,key_mlbam,key_retro_x,key_retro_y,l,last_x,last_y,ld,lev,list,mlb_played_first_x,mlb_played_first_y,mlb_played_last_x,mlb_played_last_y,moneyearned,name_first_x,name_first_y,name_last_x,name_last_y,name_x,name_y,os_avg_extension,os_avg_x,os_avg_z,os_count,os_percieved_speed,os_release_speed,os_spin_rate,os_x_movement,os_z_movement,p_throws_x,p_throws_y,pit,po,posititon,pu,r,sb,sf,so,so/w,so9,startdate,stl,str,sts,sv,team,tm,w,whip,year
0,239.0,2.0,0.0,7.0,27.0,1.0,0.500,0.0,7.0,0.0,2018-08-09,NaN,0.0,182.0,NaN,0.0,0.0,6.04946,-2.13001,5.58069,13,92.5689,93.0154,2074.85,-1.28282,0.577485,caseysadler,NaN,1.0,0.67,0.0,0.0,3.0,0.0,6.148558,-2.081228,5.778604,105,91.254548,91.644762,2022.669903,-1.411268,0.706449,0.0,0.0,0.0,NaN,1.0,sadleca02,NaN,11209.0,NaN,594987.0,sadlc002,NaN,NaN,caseysadler,NaN,0.33,MLB-NL,NaN,2014.0,NaN,2018.0,NaN,NaN,casey,NaN,sadler,NaN,caseysadler,NaN,5.61,-2.17621,5.58019,11,84.3906,85.4636,2354.82,0.161818,-0.0283273,R,R,24.0,0.0,NaN,0.0,3.0,0.0,0.0,1.0,NaN,9.0,2018-08-09,0.17,0.79,0.13,NaN,NaN,Pittsburgh,NaN,3.0,NaN
1,238.0,3.0,0.0,15.0,27.0,2.0,0.462,3.0,18.0,0.0,2018-08-10,NaN,0.0,1.0,NaN,4.0,10.8,6.04421,-2.07442,5.61094,38,91.1584,91.6737,2202.37,-1.37436,0.650429,caseysadler,NaN,1.0,0.54,2.0,0.0,6.0,0.0,6.148558,-2.081228,5.778604,105,91.254548,91.644762,2022.669903,-1.411268,0.706449,0.0,0.0,0.0,NaN,3.1,sadleca02,NaN,11209.0,NaN,594987.0,sadlc002,NaN,NaN,caseysadler,NaN,0.23,MLB-NL,NaN,2014.0,NaN,2018.0,NaN,NaN,casey,NaN,sadler,NaN,caseysadler,NaN,5.67537,-2.11583,5.61359,24,86.2358,87.0375,2448.21,0.0381625,0.458962,R,R,62.0,0.0,NaN,0.0,4.0,0.0,0.0,2.0,0.67,5.4,2018-08-10,0.16,0.58,0.05,NaN,NaN,Pittsburgh,NaN,2.7,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.805153,1.697987,5.461922,341,91.728240,92.398240,2336.364458,1.134360,1.386762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,546276.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
fb_frame = pd.DataFrame()
fb_frame = fb_frame.append(fastballs)

In [42]:
fb_frame

,key_mlbam,hist_fb_count,hist_fb_release_speed,hist_fb_percieved_speed,hist_fb_spin_rate,hist_fb_avg_x,hist_fb_avg_z,hist_fb_avg_extension,hist_fb_x_movement,hist_fb_z_movement
0,546276.0,341,92.39824,91.72824,2336.364458,1.697987,5.461922,5.805153,1.13436,1.386762


In [26]:
return_frame

,hist_fb_avg_extension,hist_fb_avg_x,hist_fb_avg_z,hist_fb_count,hist_fb_percieved_speed,hist_fb_release_speed,hist_fb_spin_rate,hist_fb_x_movement,hist_fb_z_movement,hist_os_avg_extension,hist_os_avg_x,hist_os_avg_z,hist_os_count,hist_os_percieved_speed,hist_os_release_speed,hist_os_spin_rate,hist_os_x_movement,hist_os_z_movement,key_mlbam
0,5.262154,-1.331322,5.707565,8551,86.265537,88.212583,2148.405050,-1.203380,0.998063,5.126860,-1.467546,5.665911,1624,79.950319,81.785151,1980.118786,-0.496410,0.636257,112526.0
0,6.062608,-1.569735,6.420903,444,93.090395,93.770495,2052.904878,-1.168838,1.292336,5.909676,-1.661313,6.533256,154,85.632754,86.373377,1972.307143,-0.315562,0.788430,115629.0
0,5.447056,2.621224,5.618119,54,83.061519,84.277778,2198.074074,1.356790,1.302897,4.913210,2.434560,6.014652,65,77.108500,78.690769,2206.180328,0.060359,0.575048,136600.0
0,5.968527,2.314636,5.959853,301,88.195092,88.781063,2032.142857,0.846146,1.384395,5.491444,2.328470,6.063357,318,77.195853,78.305346,2057.060498,-0.174267,0.237148,150116.0
0,5.841414,-2.136665,6.520854,74,91.106057,91.771622,2241.285714,-0.779758,1.457634,5.463596,-2.283923,6.400510,55,83.670885,84.640000,2219.897959,0.049348,0.245267,150274.0
0,6.111399,-2.504835,5.835588,459,87.671034,88.302179,1782.709677,-1.525396,0.713049,6.021669,-2.553236,5.794250,314,80.616823,80.596178,1587.302817,-0.593044,0.556129,150302.0
0,6.588302,-1.823464,5.918232,1540,91.805808,91.553182,2009.191845,-1.106436,0.910834,6.085209,-1.987720,5.849725,950,82.761909,83.247158,1941.998918,0.114306,-0.262695,150359.0
0,6.522598,0.964147,6.333970,87,84.401839,84.252874,2068.931034,0.462450,1.740010,6.448524,1.105922,6.404273,63,72.002508,72.257143,1971.523810,0.234440,-0.258962,217096.0
0,5.570074,-1.545845,5.277807,1004,87.626304,88.700000,2039.038126,-1.164730,0.601575,5.338461,-1.540103,5.242420,844,79.395071,80.620972,1956.998534,0.080744,0.032925,218596.0
0,6.013837,-1.285595,5.824657,98,89.538929,90.573469,2256.122449,-0.956494,1.323192,5.744500,-1.352730,5.877660,20,84.821250,86.055000,1900.684211,-0.300340,0.758928,234194.0


,#days,2b,3b,ab,age,appearance_yr,babip,bb,bf,cs,date,days,days_preceeding,days_rest,enddate,er,era,first_x,first_y,g,gb/fb,gdp,gs,h,hbp,hr,ibb,injured?,injury,ip,key_bbref_x,key_bbref_y,key_fangraphs_x,key_fangraphs_y,key_mlbam,key_retro_x,key_retro_y,l,last_x,last_y,ld,lev,list,mlb_played_first_x,mlb_played_first_y,mlb_played_last_x,mlb_played_last_y,moneyearned,name_first_x,name_first_y,name_last_x,name_last_y,name_x,name_y,pit,po,posititon,pu,r,sb,sf,so,so/w,so9,startdate,stl,str,sts,sv,team,tm,w,whip,year,fb_count,p_throws_x,fb_release_speed,fb_percieved_speed,fb_spin_rate,fb_avg_x,fb_avg_z,fb_avg_extension,fb_x_movement,fb_z_movement,os_count,p_throws_y,os_release_speed,os_percieved_speed,os_spin_rate,os_avg_x,os_avg_z,os_avg_extension,os_x_movement,os_z_movement
0,1460.0,0.0,0.0,22.0,42.0,1,0.154,1.0,23.0,0.0,2015-04-06,NaN,792,182.0,NaN,1.0,1.500000,bartolocolon,NaN,1.0,0.64,0.0,1.0,3.0,0.0,1.0,0.0,0.0,NaN,6.0,colonba01,NaN,375.0,NaN,112526.0,colob001,NaN,NaN,bartolocolon,NaN,0.07,MLB-NL,NaN,1997.0,NaN,2018.0,NaN,NaN,bartolo,NaN,colon,NaN,bartolocolon,NaN,86.0,0.0,NaN,0.07,1.0,0.0,0.0,8.0,8.00,12.0,2017-06-06,0.21,0.69,0.15,NaN,NaN,New York,1.0,0.667,NaN,76,R,89.7842,88.2026,2280.71,-1.48172,5.91654,5.60574,-1.02622,1.27685,10,R,83.54,81.5966,2469.44,-1.73399,5.99212,5.26267,0.0466025,0.628567
1,1454.0,1.0,1.0,27.0,42.0,2,0.238,0.0,27.0,0.0,2015-04-12,NaN,786,6.0,NaN,3.0,3.860000,bartolocolon,NaN,1.0,0.33,0.0,1.0,6.0,0.0,1.0,0.0,0.0,NaN,7.0,colonba01,NaN,375.0,NaN,112526.0,colob001,NaN,NaN,bartolocolon,NaN,0.33,MLB-NL,NaN,1997.0,NaN,2018.0,NaN,NaN,bartolo,NaN,colon,NaN,bartolocolon,NaN,77.0,0.0,NaN,0.05,3.0,0.0,0.0,5.0,NaN,6.4,2017-06-06,0.21,0.78,0.06,NaN,NaN,New York,1.0,0.857,NaN,71,R,88.569,86.3274,2133.97,-1.16116,5.9567,5.04944,-0.843298,1.23725,6,R,82,80.0865,1761.83,-1.35745,6.00008,4.996,-0.546108,0.733678
2,1449.0,2.0,0.0,26.0,42.0,3,0.250,0.0,26.0,0.0,2015-04-17,NaN,781,5.0,NaN,1.0,1.290000,bartolocolon,NaN,1.0,0.43,1.0,1.0,6.0,0.0,1.0,0.0,0.0,NaN,7.0,colonba01,NaN,375.0,NaN,112526.0,colob001,NaN,NaN,bartolocolon,NaN,0.29,MLB-NL,NaN,1997.0,NaN,2018.0,NaN,NaN,bartolo,NaN,colon,NaN,bartolocolon,NaN,91.0,0.0,NaN,0.05,1.0,0.0,0.0,5.0,NaN,6.4,2017-06-06,0.21,0.73,0.05,NaN,NaN,New York,1.0,0.857,NaN,76,R,89.2461,87.102,2193.35,-1.29515,6.12479,5.37295,-1.08747,1.03818,15,R,82.94,80.8483,1817.46,-1.52015,6.10517,5.2042,-0.449991,0.594358
3,1443.0,2.0,0.0,23.0,42.0,4,0.368,0.0,24.0,1.0,2015-04-23,NaN,775,6.0,NaN,3.0,4.500000,bartolocolon,NaN,1.0,0.22,1.0,1.0,7.0,0.0,0.0,0.0,0.0,NaN,6.0,colonba01,NaN,375.0,NaN,112526.0,colob001,NaN,NaN,bartolocolon,NaN,0.33,MLB-NL,NaN,1997.0,NaN,2018.0,NaN,NaN,bartolo,NaN,colon,NaN,bartolocolon,NaN,88.0,0.0,NaN,0.11,3.0,0.0,1.0,5.0,NaN,7.5,2017-06-06,0.23,0.66,0.07,NaN,NaN,New York,1.0,1.167,NaN,80,R,89.025,86.7799,2196.47,-1.22738,6.09687,5.24317,-1.29272,1.11685,11,R,81.9364,79.716,2036.36,-1.43826,6.18081,5.08982,-0.711843,0.584264
4,1437.0,2.0,0.0,26.0,42.0,5,0.333,0.0,29.0,0.0,2015-04-29,NaN,769,6.0,NaN,4.0,5.400000,bartolocolon,NaN,1.0,0.48,1.0,1.0,9.0,0.0,1.0,0.0,0.0,NaN,6.2,colonba01,NaN,375.0,NaN,112526.0,colob001,NaN,1.0,bartolocolon,NaN,0.28,MLB-NL,NaN,1997.0,NaN,2018.0,NaN,NaN,bartolo,NaN,colon,NaN,bartolocolon,NaN,91.0,0.0,NaN,0.04,4.0,0.0,1.0,2.0,NaN,2.7,2017-06-06,0.26,0.70,0.03,NaN,NaN,New York,NaN,1.350,NaN,72,R,88.4625,86.0994,2192.56,-0.902362,6.5241,5.15831,-1.21026,1.22752,17,R,82.4765,79.9319,2331.31,-1.20868,6.4466,4.90069,-0.0175206,0.677131
5,1431.0,0.0,0.0,29.0,42.0,6,0.263,0.0,29.0,0.0,2015-05-05,NaN,763,6.0,NaN,1.0,1.170000,bartolocolon,NaN,1.0,0.30,0.0,1.0,6.0,0.0,1.0,0.0,0.0,NaN,7.2,colonba01,NaN,375.0,NaN,112526.0,colob001,NaN,NaN,bartolocolon,NaN,0.35,MLB-NL,NaN,1997.0,NaN,2018.0,NaN,NaN,bartolo,NaN,colon,NaN,bartolocolon,NaN,97.0,0.0,NaN,0.15,1.0,0.0,0.0,9.0,NaN,10.6,2017-06-06,0.27,0.73,0.08,NaN,NaN,New York,1.0,0.783,NaN,75,R,88.4733,86.1686,2127.81,-1.32471,6.09052,5.34588,-1.25803,0.782793,22,R,82.8591,80.5772,2202.27,-1.46256,6.19927,5.15359,-0.2

In [28]:
len(id_list)

1331

In [54]:
merged = total.merge(merge1, on='key_mlbam', how='left')

In [55]:
merged

,#days,2b,3b,ab,age,appearance_yr,babip,bb,bf,cs,date,days,days_preceeding,days_rest,enddate,er,era,first_x,first_y,g,gb/fb,gdp,gs,h,hbp,hr,ibb,injured?,injury,ip,key_bbref_x,key_bbref_y,key_fangraphs_x,key_fangraphs_y,key_mlbam,key_retro_x,key_retro_y,l,last_x,last_y,ld,lev,list,mlb_played_first_x,mlb_played_first_y,mlb_played_last_x,mlb_played_last_y,moneyearned,name_first_x,name_first_y,name_last_x,name_last_y,name_x,name_y,pit,po,posititon,pu,r,sb,sf,so,so/w,so9,startdate,stl,str,sts,sv,team,tm,w,whip,year,fb_count,p_throws_x,fb_release_speed,fb_percieved_speed,fb_spin_rate,fb_avg_x,fb_avg_z,fb_avg_extension,fb_x_movement,fb_z_movement,os_count,p_throws_y,os_release_speed,os_percieved_speed,os_spin_rate,os_avg_x,os_avg_z,os_avg_extension,os_x_movement,os_z_movement,hist_fb_count,hist_fb_release_speed,hist_fb_percieved_speed,hist_fb_spin_rate,hist_fb_avg_x,hist_fb_avg_z,hist_fb_avg_extension,hist_fb_x_movement,hist_fb_z_movement,hist_os_count,hist_os_release_speed,hist_os_percieved_speed,hist_os_spin_rate,hist_os_avg_x,hist_os_avg_z,hist_os_avg_extension,hist_os_x_movement,hist_os_z_movement
0,1460.0,0.0,0.0,22.0,42.0,1,0.154,1.0,23.0,0.0,2015-04-06,NaN,792,182.0,NaN,1.0,1.500000,bartolocolon,NaN,1.0,0.64,0.0,1.0,3.0,0.0,1.0,0.0,0.0,NaN,6.0,colonba01,NaN,375.0,NaN,112526.0,colob001,NaN,NaN,bartolocolon,NaN,0.07,MLB-NL,NaN,1997.0,NaN,2018.0,NaN,NaN,bartolo,NaN,colon,NaN,bartolocolon,NaN,86.0,0.0,NaN,0.07,1.0,0.0,0.0,8.0,8.00,12.0,2017-06-06,0.21,0.69,0.15,NaN,NaN,New York,1.0,0.667,NaN,76,R,89.7842,88.2026,2280.71,-1.48172,5.91654,5.60574,-1.02622,1.27685,10,R,83.54,81.5966,2469.44,-1.73399,5.99212,5.26267,0.0466025,0.628567,8551,88.212583,86.265537,2148.405050,-1.331322,5.707565,5.262154,-1.203380,0.998063,1624,81.785151,79.950319,1980.118786,-1.467546,5.665911,5.126860,-0.496410,0.636257
1,1454.0,1.0,1.0,27.0,42.0,2,0.238,0.0,27.0,0.0,2015-04-12,NaN,786,6.0,NaN,3.0,3.860000,bartolocolon,NaN,1.0,0.33,0.0,1.0,6.0,0.0,1.0,0.0,0.0,NaN,7.0,colonba01,NaN,375.0,NaN,112526.0,colob001,NaN,NaN,bartolocolon,NaN,0.33,MLB-NL,NaN,1997.0,NaN,2018.0,NaN,NaN,bartolo,NaN,colon,NaN,bartolocolon,NaN,77.0,0.0,NaN,0.05,3.0,0.0,0.0,5.0,NaN,6.4,2017-06-06,0.21,0.78,0.06,NaN,NaN,New York,1.0,0.857,NaN,71,R,88.569,86.3274,2133.97,-1.16116,5.9567,5.04944,-0.843298,1.23725,6,R,82,80.0865,1761.83,-1.35745,6.00008,4.996,-0.546108,0.733678,8551,88.212583,86.265537,2148.405050,-1.331322,5.707565,5.262154,-1.203380,0.998063,1624,81.785151,79.950319,1980.118786,-1.467546,5.665911,5.126860,-0.496410,0.636257
2,1449.0,2.0,0.0,26.0,42.0,3,0.250,0.0,26.0,0.0,2015-04-17,NaN,781,5.0,NaN,1.0,1.290000,bartolocolon,NaN,1.0,0.43,1.0,1.0,6.0,0.0,1.0,0.0,0.0,NaN,7.0,colonba01,NaN,375.0,NaN,112526.0,colob001,NaN,NaN,bartolocolon,NaN,0.29,MLB-NL,NaN,1997.0,NaN,2018.0,NaN,NaN,bartolo,NaN,colon,NaN,bartolocolon,NaN,91.0,0.0,NaN,0.05,1.0,0.0,0.0,5.0,NaN,6.4,2017-06-06,0.21,0.73,0.05,NaN,NaN,New York,1.0,0.857,NaN,76,R,89.2461,87.102,2193.35,-1.29515,6.12479,5.37295,-1.08747,1.03818,15,R,82.94,80.8483,1817.46,-1.52015,6.10517,5.2042,-0.449991,0.594358,8551,88.212583,86.265537,2148.405050,-1.331322,5.707565,5.262154,-1.203380,0.998063,1624,81.785151,79.950319,1980.118786,-1.467546,5.665911,5.126860,-0.496410,0.636257
3,1443.0,2.0,0.0,23.0,42.0,4,0.368,0.0,24.0,1.0,2015-04-23,NaN,775,6.0,NaN,3.0,4.500000,bartolocolon,NaN,1.0,0.22,1.0,1.0,7.0,0.0,0.0,0.0,0.0,NaN,6.0,colonba01,NaN,375.0,NaN,112526.0,colob001,NaN,NaN,bartolocolon,NaN,0.33,MLB-NL,NaN,1997.0,NaN,2018.0,NaN,NaN,bartolo,NaN,colon,NaN,bartolocolon,NaN,88.0,0.0,NaN,0.11,3.0,0.0,1.0,5.0,NaN,7.5,2017-06-06,0.23,0.66,0.07,NaN,NaN,New York,1.0,1.167,NaN,80,R,89.025,86.7799,2196.47,-1.22738,6.09687,5.24317,-1.29272,1.11685,11,R,81.9364,79.716,2036.36,-1.43826,6.18081,5.08982,-0.711843,0.584264,8551,88.212583,86.265537,2148.405050,-1.331322,5.707565,5.262154,-1.203380,0.998063,1624,81.785151,79.950319,1980.118786,-1.467546,5.665911,5.126860,-0.496410,0.636257
4,1437.0,2.0,0.0,26.0,42.0,5,0.333,0.0,29.0,0.0,2015-04-29,NaN,76

In [57]:
merged[merged['hist_fb_count'].isnull() & merged['hist_os_count'].isnull()]

,#days,2b,3b,ab,age,appearance_yr,babip,bb,bf,cs,date,days,days_preceeding,days_rest,enddate,er,era,first_x,first_y,g,gb/fb,gdp,gs,h,hbp,hr,ibb,injured?,injury,ip,key_bbref_x,key_bbref_y,key_fangraphs_x,key_fangraphs_y,key_mlbam,key_retro_x,key_retro_y,l,last_x,last_y,ld,lev,list,mlb_played_first_x,mlb_played_first_y,mlb_played_last_x,mlb_played_last_y,moneyearned,name_first_x,name_first_y,name_last_x,name_last_y,name_x,name_y,pit,po,posititon,pu,r,sb,sf,so,so/w,so9,startdate,stl,str,sts,sv,team,tm,w,whip,year,fb_count,p_throws_x,fb_release_speed,fb_percieved_speed,fb_spin_rate,fb_avg_x,fb_avg_z,fb_avg_extension,fb_x_movement,fb_z_movement,os_count,p_throws_y,os_release_speed,os_percieved_speed,os_spin_rate,os_avg_x,os_avg_z,os_avg_extension,os_x_movement,os_z_movement,hist_fb_count,hist_fb_release_speed,hist_fb_percieved_speed,hist_fb_spin_rate,hist_fb_avg_x,hist_fb_avg_z,hist_fb_avg_extension,hist_fb_x_movement,hist_fb_z_movement,hist_os_count,hist_os_release_speed,hist_os_percieved_speed,hist_os_spin_rate,hist_os_avg_x,hist_os_avg_z,hist_os_avg_extension,hist_os_x_movement,hist_os_z_movement
81313,232.0,2.0,0.0,8.0,24.0,1,0.5,0.0,8.0,0.0,2018-08-16,NaN,0,182.0,NaN,2.0,13.5,scottkingery,NaN,1.0,0.5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,NaN,1.1,kingesc01,NaN,17975.0,NaN,664068.0,kings001,NaN,NaN,scottkingery,NaN,0.5,MLB-NL,NaN,2018.0,NaN,2019.0,NaN,NaN,scott,NaN,kingery,NaN,scottkingery,NaN,16.0,0.0,NaN,0.0,2.0,0.0,0.0,0.0,NaN,0.0,2018-08-16,0.25,0.81,0.0,NaN,NaN,Philadelphia,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
merged = merged[merged['key_mlbam'] != 664068.0]

In [61]:
merged = merged.set_index(np.arange(len(merged)))

In [63]:
merged.to_pickle('../pickles/labeled_games_agg_sc_hist.pkl')